# Comparing SSN with the corresponding linearly approximated models and the corresponding linear Hawkes models

In [ ]:
# Installation
using Revise
using LinearAlgebra,Statistics,StatsBase,Distributions
using Plots,NamedColors ; theme(:default)
using FFTW
using ProgressMeter
using Random
Random.seed!(0)
using HawkesSimulator; const global H = HawkesSimulator

In [ ]:
function onedmat(x::Real)
  return cat(x;dims=2)
end

In [ ]:
function rate_powerlaw(v::Float64, v_rest::Real, n::Real, k::Real)
    diff = v - v_rest
    diff = (diff < 0) ? 0 : diff
    return k*(diff^n)
end

In [ ]:
# Euler Method
function dv_ssn(v::Float64, v_rest::Real, h_i::Float64, noise_i::Float64, tau_i::Float64, w_arr::Vector{Float64}, rate_arr::Vector{Float64})
    num_neurons = size(w_arr, 1)
    dv = h_i - v + v_rest
    for i in 1:num_neurons
        dv += w_arr[i]*rate_arr[i]
    end
    return dv*time_step/tau_i
end

In [ ]:
function simulate_with_v!(h, n, w, k, num_steps, t_arr, v_excite, v_excite_rest, v_inhibit, v_inhibit_rest, rate, tau)
    
    @showprogress 1.0 "Running SSN simulation..." for i in 2:num_steps
        
        rate[i,:] = [rate_powerlaw(v_excite[i-1], v_excite_rest, n, k) , rate_powerlaw(v_inhibit[i-1], v_inhibit_rest, n, k)]
        v_excite[i] = v_excite[i-1] + dv_ssn(v_excite[i-1], v_excite_rest, h[1], noise[1], tau[1], [w[1,1], w[1,2]], rate[i,:])
        v_inhibit[i] = v_inhibit[i-1] + dv_ssn(v_inhibit[i-1], v_inhibit_rest, h[2], noise[2], tau[2], [w[2,1], w[2,2]], rate[i,:])
        t_arr[i] = t_arr[i-1] + time_step
    end
    
end

In [ ]:
function create_parameters_for_approximation!(w_linear, h_linear, w_ssn, k_ssn, h_ssn, rate_ssn_stable::Vector{Float64})
    
    w_linear[1,1] = 3*w_ssn[1,1]*sqrt(k_ssn*rate_ssn_stable[1]) + sqrt(k_ssn/rate_ssn_stable[1])*(h_ssn[1]+w_ssn[1,2]*rate_ssn_stable[2])
    w_linear[1,2] = 2*w_ssn[1,2]*sqrt(k_ssn*rate_ssn_stable[1])
    w_linear[2,1] = 2*w_ssn[2,1]*sqrt(k_ssn*rate_ssn_stable[2])
    w_linear[2,2] = 3*w_ssn[2,2]*sqrt(k_ssn*rate_ssn_stable[2]) + sqrt(k_ssn/rate_ssn_stable[2])*(h_ssn[2]+w_ssn[2,1]*rate_ssn_stable[1])
    
    
    h_linear[1] = (2-w_linear[1,1])*rate_ssn_stable[1] - w_linear[1,2]*rate_ssn_stable[2]
    h_linear[2] = (2-w_linear[2,2])*rate_ssn_stable[2] - w_linear[2,1]*rate_ssn_stable[1]

end

In [ ]:
# Euler Method
function dr_linear(r::Real, h_i::Real, tau_i::Real, w_arr::Vector{Float64}, rate_arr::Vector{Float64})
    num_neurons = size(w_arr, 1)
    dr = h_i - 2*r
    for i in 1:num_neurons
        dr += w_arr[i]*rate_arr[i]
    end
    return dr*time_step/tau_i
end

In [ ]:
function simulate_linear_with_r!(h, w, tau, num_steps, t_arr, rate)
    
    @showprogress 1.0 "Running linear simulation..." for i in 2:num_steps
        
        rate[i,1] = rate[i-1,1] + dr_linear(rate[i-1,1], h[1], tau[1], [w[1,1], w[1,2]], rate[i-1,:])
        rate[i,2] = rate[i-1,2] + dr_linear(rate[i-1,2], h[2], tau[2], [w[2,1], w[2,2]], rate[i-1,:])
        t_arr[i] = t_arr[i-1] + time_step
        
    end
    
end

In [ ]:
function simulate_hawkes!(networks, num_spikes)
    t_now = 0.0
    H.reset!.(networks) # clear spike trains etc
    for k in 1:num_spikes
        t_now = H.dynamics_step!(t_now, networks)
        if k%1_000 == 0
            H.clear_trains!(networks[1].postpops)
            H.clear_trains!(networks[2].postpops)
        end # clearing trains after every 1000 spikes
    end
    return t_now
end

In [ ]:
function compare_single_h(h_ssn)
    t_max = 1
    num_steps = Int(ceil(t_max/time_step))
    t_arr = zeros(num_steps)
    
#    # SSN
    v_excite_ssn = zeros(num_steps)
    v_inhibit_ssn = zeros(num_steps)
    v_excite_ssn[1] = v_rest
    v_inhibit_ssn[1] = v_rest
    rate_ssn = zeros(num_steps, 2)
    h = [h_ssn, h_ssn]
    simulate_with_v!(h, n_ssn, w_ssn, k_ssn, num_steps, t_arr, v_excite_ssn, v_rest, v_inhibit_ssn, v_rest, rate_ssn, tau_ssn)
    
#     # Linear
    w_linear = [0.0 0.0
                0.0 0.0]
    h_linear = zeros(2)
    create_parameters_for_approximation!(w_linear, h_linear, w_ssn, k_ssn, h, rate_ssn[num_steps,:])
    rate_linear = zeros(num_steps, 2)
    simulate_linear_with_r!(h_linear, w_linear, tau_linear, num_steps, t_arr, rate_linear)
    
#     # Hawkes
    w_hawkes = w_linear ./ 2
    baseline_rate = h_linear ./ 2
    # this division by 2 is essentially taking a common factor of 2 out from eq. 3.22 of the report,
    # this was not done in the linear case to avoid working with extremely small numbers,
    # but cannot be ignored for the hawkes case to be able to use to previous code from the linear rate model and linear hawkes comparison
    n_spikes = 500_000
    tau_E = 10.0
    tau_I = 5.0
    pop_E = H.PopulationExp(tau_E, H.NLIdentity())
    pop_I = H.PopulationExp(tau_I, H.NLIdentity())
    popstate_E = H.PopulationState(pop_E,[baseline_rate[1]])
    popstate_I = H.PopulationState(pop_I, [baseline_rate[2]])
    network_E = H.InputNetwork(popstate_E,[popstate_E, popstate_I],[onedmat(w_hawkes[1,1]),onedmat(w_hawkes[1,2])])
    network_I = H.InputNetwork(popstate_I,[popstate_E, popstate_I],[onedmat(w_hawkes[2,1]),onedmat(w_hawkes[2,2])])
    simulate_hawkes!([network_E, network_I],n_spikes)
    rate_hawkes_linear = [H.numerical_rate(popstate_E.trains_history[1][1000:end]), H.numerical_rate(popstate_I.trains_history[1][1000:end])]
    return rate_ssn, rate_linear, rate_hawkes_linear, v_excite_ssn, v_inhibit_ssn, t_arr
end

In [ ]:
# Initializing SSN variables
# one E, one I population

v_rest = -70
tau_ssn = [20.0, 10.0]*1E-3   #time constant, [ E, I ]
tau_linear = [10.0, 5.0]*1E-3
time_step = 0.1*1E-3
noise = [0.0 0.0] #ignoring

k_ssn = 0.3
n_ssn = 2
w_ssn = [1.25 -0.65
         1.2 -0.5]
h_ssn = 12.0
rate_ssn, rate_linear, rate_hawkes_linear, v_excite_ssn, v_inhibit_ssn, t_arr = compare_single_h(h_ssn)
println(rate_ssn[end,:])
println(rate_linear[end,:])
println(rate_hawkes_linear)

In [ ]:
plt = plot( xlabel="time (s)", ylabel="E Rate (Hz)", fmt=:png, legend=:bottomright)
plot!(plt, t_arr , rate_ssn[:,1], label = "ssn", color="blue")
plot!(plt, t_arr , rate_linear[:,1], label = "linear", color="dark blue")
rate_hawkes_linear_arr = zeros(length(t_arr),2)
for i in 1:length(t_arr)
    rate_hawkes_linear_arr[i,1] = rate_hawkes_linear[1]
    rate_hawkes_linear_arr[i,2] = rate_hawkes_linear[2]
end
plot!(plt, t_arr , rate_hawkes_linear_arr[:,1], label = "hawkes", color="purple", linestyle=:dash)

In [ ]:
plt = plot( xlabel="time (s)", ylabel="I Rate (Hz)", fmt=:png, legend=:bottomright)
plot!(plt, t_arr , rate_ssn[:,2], label = "ssn", color="red")
plot!(plt, t_arr , rate_linear[:,2], label = "linear", color="dark red")
plot!(plt, t_arr , rate_hawkes_linear_arr[:,2], label = "hawkes", color="black", linestyle=:dash)

In [ ]:
# Extending for multiple h values
function compare()
    
    for i in 2:num_h
        h_ssn[i,:] = h_ssn[i-1,:] .+ step
    end
    
    @showprogress 1.0 "Running for multiple h..." for i in 1:num_h
        t_max = 1
        num_steps = Int(ceil(t_max/time_step))
        t_arr = zeros(num_steps)
        # SSN
        v_excite_ssn = zeros(num_steps)
        v_inhibit_ssn = zeros(num_steps)
        rate_ssn = zeros(num_steps, 2)
        simulate_with_v!(h_ssn[i,:], n_ssn, w_ssn, k_ssn, num_steps, t_arr, v_excite_ssn, v_rest, v_inhibit_ssn, v_rest, rate_ssn, tau_ssn)
        rates_ssn[i,:] = rate_ssn[num_steps,:]
        # Linear
        w_linear = [0.0 0.0
                    0.0 0.0]
        h_linear = zeros(2)
        create_parameters_for_approximation!(w_linear, h_linear, w_ssn, k_ssn, h_ssn, rate_ssn[num_steps,:])
        rate_linear = zeros(num_steps, 2)
        simulate_linear_with_r!(h_linear, w_linear, tau_linear, num_steps, t_arr, rate_linear)
        rates_linear[i,:] = rate_linear[num_steps,:]
        # Hawkes
        w_hawkes = w_linear ./ 2
        baseline_rate = h_linear ./ 2
        n_spikes = 500_000
        tau_E = 10.0
        tau_I = 5.0
        pop_E = H.PopulationExp(tau_E, H.NLIdentity())
        pop_I = H.PopulationExp(tau_I, H.NLIdentity())
        popstate_E = H.PopulationState(pop_E,[baseline_rate[1]])
        popstate_I = H.PopulationState(pop_I, [baseline_rate[2]])
        network_E = H.InputNetwork(popstate_E,[popstate_E, popstate_I],[onedmat(w_hawkes[1,1]),onedmat(w_hawkes[1,2])])
        network_I = H.InputNetwork(popstate_I,[popstate_E, popstate_I],[onedmat(w_hawkes[2,1]),onedmat(w_hawkes[2,2])])
        simulate_hawkes!([network_E, network_I],n_spikes)
        rates_hawkes_linear[i,:] = [H.numerical_rate(popstate_E.trains_history[1][1000:end]), H.numerical_rate(popstate_I.trains_history[1][1000:end])]
    end
end

In [ ]:
# Initializing SSN variables
# one E, one I population

v_rest = -70
tau_ssn = [20.0, 10.0]*1E-3   #time constant, [ E, I ]
tau_linear = [10.0, 5.0]*1E-3
time_step = 0.1*1E-2
noise = [0.0 0.0] #ignoring

k_ssn = 0.3
n_ssn = 2
w_ssn = [1.25 -0.65
         1.2 -0.5]

In [ ]:
h_max = 20.0
h_min = 1.0
step = 1
num_h = Int(ceil((h_max-h_min+1)/step))
h_ssn = zeros(num_h,2)
h_ssn[1,:] = [h_min, h_min]
rates_ssn = zeros(num_h,2)
rates_linear = zeros(num_h,2)
rates_hawkes_linear = zeros(num_h,2)

compare()

In [ ]:
plt = plot( xlabel="h (mV)", ylabel="E Rate (Hz)", fmt=:png, legend=:bottomright)
plot!(plt, h_ssn[:,1] , rates_ssn[:,1], label = "ssn")
plot!(plt, h_ssn[:,1] , rates_linear[:,1], label = "linear")
plot!(plt, h_ssn[:,1] , rates_hawkes_linear[:,1], label = "hawkes")

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="I Rate (Hz)", fmt=:png, legend=:bottomright)
plot!(plt, h_ssn[:,1] , rates_ssn[:,2], label = "ssn")
plot!(plt, h_ssn[:,1] , rates_linear[:,2], label = "linear")
plot!(plt, h_ssn[:,1] , rates_hawkes_linear[:,2], label = "hawkes")

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 1:num_h
    y_e[i] = 2*abs(rates_hawkes_linear[i,1] - rates_linear[i,1])/(rates_hawkes_linear[i,1] + rates_linear[i,1])
    y_i[i] = 2*abs(rates_hawkes_linear[i,2] - rates_linear[i,2])/(rates_hawkes_linear[i,2] + rates_linear[i,2])
end
plt = plot(xlabel = "h", ylabel="relative error", fmt=:png)
plot!(plt, h_ssn[:,1], y_e, label="E", color="blue")
plot!(plt, h_ssn[:,1], y_i, label="I", color="red")

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 1:num_h
    y_e[i] = abs(rates_hawkes_linear[i,1] - rates_linear[i,1])
    y_i[i] = abs(rates_hawkes_linear[i,2] - rates_linear[i,2])
end
plt = plot(xlabel = "h", ylabel="absolute error", fmt=:png)
plot!(plt, h_ssn[:,1], y_e, label="E", color="blue")
plot!(plt, h_ssn[:,1], y_i, label="I", color="red")